In [ ]:
import os
import sys

In [ ]:
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'
spark_home = os.environ.get('SPARK_HOME', None)

if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json
from pyspark import SparkContext
conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [ ]:
schema = StructType(fields=[
    StructField("lang", StringType()),
    StructField("name", StringType()),
    StructField("cat", StringType()),
    StructField("id", IntegerType()),
    StructField("desc", StringType())
])

In [ ]:
df = spark.read.json("/labs/slaba02/DO_record_per_line.json", schema)

In [ ]:
df.show(5)

In [ ]:
def change_chars(str_1):
    regex = re.compile(u'[\w\d]{2,}', re.U)
    
    return regex.findall(str_1.lower())

In [ ]:
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType,IntegerType,StringType
change_charsUDF = udf(lambda z:change_chars(z),StringType())
# df = df.withColumn("desc_new", change_charsUDF("desc")) 
df = df.select("id", \
    change_charsUDF("desc").alias("desc") )

In [ ]:
df.show()

In [ ]:
from pyspark.ml.feature import RegexTokenizer, Tokenizer
# retokenizer = RegexTokenizer(inputCol="desc", outputCol="words", pattern="[0-9]{5,}")
# df = retokenizer.transform(df)
tokenizer = Tokenizer(inputCol="desc",outputCol="words")
# df = tokenizer.transform(df)
# df.select('words').show()

In [ ]:
from pyspark.ml import Pipeline
preprocessing = Pipeline(stages=[
    tokenizer])

In [ ]:
preprocessing_model = preprocessing.fit(df)

In [ ]:
preprocessed_dataset = preprocessing_model.transform(df)

In [ ]:
preprocessed_dataset.select('id','words').where('id = 23126').collect()

In [ ]:
preprocessed_dataset.select('words').show()

In [ ]:
from pyspark.ml.feature import HashingTF
ht = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000)

In [ ]:
result_tf = ht.transform(preprocessed_dataset)
# result_tf.select('rawFeatures').show()

In [ ]:
from pyspark.ml.feature import IDF

In [ ]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(result_tf)
rescaledData = idfModel.transform(result_tf)

In [ ]:
rescaledData.select('id','features').show()

In [ ]:
my_id_list = [23126, 21617, 16627, 11556, 16704, 13702]
# rescaledData.select('id','features').where('id = 23126').show()

In [ ]:
only_ids = rescaledData.select('id')
# only_ids.show()

In [ ]:
all_ids=only_ids.rdd.map(lambda x: x[0]).collect()
print(all_ids)

In [ ]:
only_tfidf = rescaledData.select('features')
# only_tfidf.show()

In [ ]:
# all_tfidf = only_tfidf.rdd.map(lambda x: x[0]).collect()
all_tfidf = only_tfidf.collect()
all_tfidf

In [ ]:
all_tfidf = only_tfidf.rdd.map(lambda x: x[0]).collect()
all_tfidf[1]

In [ ]:
type(all_tfidf[1])

In [ ]:
print(all_tfidf[1].dot(all_tfidf[2]) / (all_tfidf[1].norm(2) * all_tfidf[2].norm(2)))

In [ ]:
dict_cos_sim = {}
for l in my_id_list:
    dict_cos_sim[l] = []

In [ ]:
def func_get_cos_sim(my_id, list_of_all_ids,all_tfidf):
    dict_output = {}
    my_id_index = list_of_all_ids.index(my_id)
    for i in list_of_all_ids:
        if my_id != i:
            index_of_el = list_of_all_ids.index(i)
            dict_output[i] = all_tfidf[my_id_index].dot(all_tfidf[index_of_el]) / (all_tfidf[my_id_index].norm(2) * all_tfidf[index_of_el].norm(2))
    return dict_output

In [ ]:
dict_sorted = {}
for i,v in enumerate(my_id_list):
    dict_1 = func_get_cos_sim(v,all_ids, all_tfidf)
    sorted_tuple = sorted(dict_1.items(), key=lambda x: x[1], reverse = True)
    top_10 = sorted_tuple[0:10]
    anwr = []
    for k in top_10:
        anwr.append(k[0])
    dict_sorted[str(v)] = anwr

In [ ]:
dict_sorted

In [ ]:
df.select('desc').where('id = 16627').collect()

In [ ]:
asd = [25010, 387, 386, 166, 196, 59, 160, 198, 468, 2474]
df.select('desc').where('id = 166').collect()

In [ ]:
df.select('desc').where('id = 196').collect()

In [ ]:
preprocessed_dataset.select('words').collect()

In [ ]:
import json

with open('lab02.json', "w") as file:
    json.dump(dict_sorted, file)

In [ ]:
spark.stop()